In [1]:
import torch
from diffusers import FluxPipeline

/Users/twinsant/GitHub/ML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Apple Silicon (M5) 优化配置
# 32GB统一内存应该足够，但需要正确配置MPS

import gc

# 清理内存
gc.collect()

# 检查MPS可用性
if torch.backends.mps.is_available():
    device = "mps"
    print("Using Apple Silicon MPS acceleration")
elif torch.cuda.is_available():
    device = "cuda"
    print("Using CUDA")
else:
    device = "cpu"
    print("Using CPU only")

# 加载模型 - 使用float16而不是bfloat16（MPS对float16支持更好）
pipe = FluxPipeline.from_pretrained(
    '/Users/twinsant/models/FLUX.1-dev',
    torch_dtype=torch.float16,  # MPS推荐使用float16
)

# 移动到MPS设备
pipe = pipe.to(device)

# 启用VAE优化（减少内存峰值）
pipe.enable_vae_slicing()
pipe.enable_vae_tiling()

# 注意：MPS不支持CPU offload，所以不使用enable_model_cpu_offload()

print(f"Pipeline loaded on {device} with memory optimizations")

Using Apple Silicon MPS acceleration


Loading pipeline components...:  14%|█▍        | 1/7 [00:14<01:26, 14.37s/it]

: 

In [3]:
# Apple Silicon 32GB - 使用CPU模式避免MPS内存问题
import gc

# 清理内存
gc.collect()

print("Loading FLUX.1-dev in CPU mode (MPS causes OOM on 32GB)")
print("This will be slower but won't crash...")

# 完全在CPU上加载和运行，避免MPS的内存管理问题
pipe = FluxPipeline.from_pretrained(
    '/Users/twinsant/models/FLUX.1-dev',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,  # 关键：低内存模式
)

# 保持在CPU上，不移动到MPS
# MPS在加载大模型时会导致内存峰值过高
print("Keeping model on CPU to avoid memory spikes")

# 启用所有内存优化
pipe.enable_vae_slicing()
pipe.enable_vae_tiling()

# 启用注意力切片
try:
    pipe.enable_attention_slicing(1)
    print("Attention slicing enabled")
except:
    pass

print("✓ Pipeline loaded successfully on CPU")
print("Note: Generation will be slow (~10-30 minutes) but won't crash")

Loading FLUX.1-dev in CPU mode (MPS causes OOM on 32GB)
This will be slower but won't crash...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

: 

In [1]:
# 生成图像 - 降低配置避免崩溃
prompt = "A cat holding a sign that says 你好 world"

# 清理内存
gc.collect()

print("Starting image generation with reduced settings...")

# 使用更保守的参数
image = pipe(
    prompt,
    height=512,  # 降低到512避免内存峰值
    width=512,
    guidance_scale=3.5,
    num_inference_steps=20,  # 减少步骤
    max_sequence_length=256,  # 减少序列长度
    generator=torch.Generator("cpu").manual_seed(0)  # 使用CPU generator
).images[0]

# 保存结果
output_path = "output.png"
image.save(output_path)
print(f"✓ Image generated and saved as {output_path}")

# 显示图像
image

NameError: name 'gc' is not defined